In [3]:
import pandas as pd
import os
from glob import glob
import ast
import re
import spacy
import time

In [2]:
#additional nlp models
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 2.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [4]:
# Access dataset files
folder_path = os.path.join( "..", "Data") 
file_type = "*.csv"

# List of dataset file paths
document_path = glob(os.path.join(folder_path, file_type))

document_path

['../Data/subset_sample_no_label.csv',
 '../Data/filtered_corpus.csv',
 '../Data/full_posts.csv',
 '../Data/full_data_no_preprocessing.csv',
 '../Data/training_data.csv',
 '../Data/lda_train.csv',
 '../Data/vocabulary.csv',
 '../Data/emoji_subset.csv',
 '../Data/sampled_subset.csv',
 '../Data/subs_topics.csv',
 '../Data/comments.csv']

In [5]:
data_filepath = '' #file path for csv file of scraped comments 

for path in document_path:
    if "subset_sample" in path:
        data_filepath = data_filepath + path

print (data_filepath)
    

../Data/subset_sample_no_label.csv


In [6]:
#import cleaned data

def list_converter(text):
    #to revert list->str conversion from pd.read_csv
    return ast.literal_eval(text)


test_data = pd.read_csv(data_filepath, converters ={'tokens':list_converter})
test_data = test_data.drop(columns = ['index', 'clean_text', 'tokens', 'word_count'])

In [7]:
#sort by data created
test_data.sort_values(by='date_created', inplace = True, ignore_index = True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text_type     1200 non-null   object
 1   ID            1200 non-null   object
 2   date_created  1200 non-null   object
 3   year          1200 non-null   int64 
 4   long_text     1200 non-null   object
dtypes: int64(1), object(4)
memory usage: 47.0+ KB


In [8]:
test_data.head()

,text_type,ID,date_created,year,long_text
0,submission,wbwik,2012-07-10 19:11:08,2012,Only in Dubai! R8: BRAND NEW 2012 Audi R8 V8 -...
1,comment,c5c2v3q,2012-07-10 22:36:50,2012,"Surprise! Happy birthday! \n\nAh man, *anoth..."
2,comment,c5cc9sz,2012-07-11 08:48:34,2012,I looked at the AD and it seems more like a sa...
3,comment,c5cdavh,2012-07-11 10:15:52,2012,It's a marketing post. But isn't it against du...
4,submission,10embj,2012-09-24 23:02:05,2012,Dubai cap my dad brought me! [FYSR]


In [20]:
#remove URls

In [9]:
html_index = test_data[test_data['long_text'].str.contains("https")].index
test_data.loc[html_index].head()

,text_type,ID,date_created,year,long_text
177,comment,d097zee,2016-02-22 17:15:18,2016,Oi [resident r/Dubai hipster](https://www.redd...
253,comment,d9yxdx9,2016-11-14 01:31:36,2016,So you know classic SNL? https://youtu.be/toji...
321,comment,dm9vlct,2017-08-29 13:18:31,2017,Ok this fantastic piece of content just went u...
323,comment,dm9vtv0,2017-08-29 13:29:37,2017,>Maximusbeatdown: Important enough for you to ...
353,comment,dnstajv,2017-10-02 18:00:25,2017,I usually send a removal comment when I delete...


In [10]:
#regex pattern for urls
url_pattern = r'https?://\S+'
#replace url with empty string
test_data['clean_text'] = test_data['long_text'].apply(lambda text: re.sub(url_pattern, ' ', text, flags=re.MULTILINE))

test_data.loc[html_index].head()

,text_type,ID,date_created,year,long_text,clean_text
177,comment,d097zee,2016-02-22 17:15:18,2016,Oi [resident r/Dubai hipster](https://www.redd...,Oi [resident r/Dubai hipster]( what choice ar...
253,comment,d9yxdx9,2016-11-14 01:31:36,2016,So you know classic SNL? https://youtu.be/toji...,So you know classic SNL?
321,comment,dm9vlct,2017-08-29 13:18:31,2017,Ok this fantastic piece of content just went u...,Ok this fantastic piece of content just went u...
323,comment,dm9vtv0,2017-08-29 13:29:37,2017,>Maximusbeatdown: Important enough for you to ...,>Maximusbeatdown: Important enough for you to ...
353,comment,dnstajv,2017-10-02 18:00:25,2017,I usually send a removal comment when I delete...,I usually send a removal comment when I delete...


In [11]:
#remove newline & tab
#remove `\n` from text
test_data['clean_text'] = test_data['clean_text'].str.replace('\n', ' ')
#remove `\t` from text
test_data['clean_text'] = test_data['clean_text'].str.replace('\t', ' ')

test_data.sample(n = 5)

,text_type,ID,date_created,year,long_text,clean_text
1182,comment,jom3ot0,2023-06-18 22:37:14,2023,"From my personal experience, when you visit th...","From my personal experience, when you visit th..."
958,comment,if9924s,2022-07-08 00:52:23,2022,That’s the best neighborhood in the city ❤️ Sa...,That’s the best neighborhood in the city ❤️ Sa...
794,comment,gvg8gi5,2021-04-22 18:57:31,2021,This is fucking hilarious and so try! I’ve sta...,This is fucking hilarious and so try! I’ve sta...
727,comment,gbkat5m,2020-11-08 09:41:56,2020,I guess KT and GN will have to struggle to fin...,I guess KT and GN will have to struggle to fin...
1027,comment,isnnolm,2022-10-17 14:11:58,2022,"Rape cannot be reported anonymously, are you n...","Rape cannot be reported anonymously, are you n..."


In [12]:
#lemmatize text
nlp = spacy.load('en_core_web_md')

start_time = time.time()

test_data['clean_text'] = test_data['clean_text'].apply(lambda text:
                                              ' '.join(token.lemma_ for token in nlp(text)))

print (f"This code block took {(time.time() - start_time)/60 :.2f} minutes to complete")

test_data.sample(n=5)

This code block took 0.30 minutes to complete


,text_type,ID,date_created,year,long_text,clean_text
298,submission,6s6kno,2017-08-07 20:14:52,2017,Does anyone here Believe in ghosts?Any Encoute...,do anyone here believe in ghosts?any Encouters...
851,comment,h9dxkut,2021-08-18 11:55:43,2021,"Same I was just hoping to scale sooner, As my ...","same I be just hope to scale soon , as my daug..."
408,comment,duaof8u,2018-02-15 22:01:10,2018,Should've just driven to that patch on SZR nex...,should 've just drive to that patch on SZR nex...
830,comment,h4fxkbn,2021-07-08 09:15:26,2021,Seriously. Lebanon's wiped out part of the cit...,seriously . Lebanon 's wipe out part of the ci...
820,comment,h3e6e9h,2021-06-29 06:03:55,2021,"Please look up Hypnobirthing, it sounds like h...","please look up Hypnobirthing , it sound like h..."


In [13]:
test_data.to_csv('../bert_model_data/test_data_lemma.csv')

In [14]:
# Access dataset files
folder_path = os.path.join( "..", "bert_model_data") 
file_type = "*.csv"

# List of dataset file paths
document_path = glob(os.path.join(folder_path, file_type))

document_path

['../bert_model_data/bert_default_raw_text_topics_term.csv',
 '../bert_model_data/test_data_bert_raw_label.csv',
 '../bert_model_data/bert_train_lemma.csv',
 '../bert_model_data/human_and_bert_raw_labels.csv',
 '../bert_model_data/bert_default_raw_text_topics_term_labelled.csv',
 '../bert_model_data/bert_kmeans_lemma_topics_terms.csv',
 '../bert_model_data/test_data_human_label_bert_raw.csv',
 '../bert_model_data/posts_upvote_ratio.csv',
 '../bert_model_data/bert_train_no_prep.csv',
 '../bert_model_data/bert_model_1_label_train_data.csv',
 '../bert_model_data/test_data_lemma.csv']

In [15]:
data_filepath = '' #file path for csv file of scraped comments 

for path in document_path:
    if "no_prep" in path:
        data_filepath = data_filepath + path

print (data_filepath)
    

../bert_model_data/bert_train_no_prep.csv


In [18]:
bert_data = pd.read_csv(data_filepath, converters ={'tokens':list_converter})
bert_data.drop(columns = ['index'], inplace = True)

In [19]:
bert_data.head()

,text_type,ID,date_created,year,long_text
0,comment,gtfo2hl,2021-04-05 13:00:32,2021,"*Cuntry roads, take me hoem*"
1,comment,gtfqkbv,2021-04-05 13:41:40,2021,"That’s been there for several years, sent a pi..."
2,comment,gtfou07,2021-04-05 13:13:23,2021,I am single and I have not traveled to any cun...
3,comment,gtfrgpe,2021-04-05 13:56:09,2021,What happens when you shop at dragon mart...
4,comment,gtg5mwv,2021-04-05 16:51:54,2021,I am cunting on them to do so 😅


In [20]:
bert_data.sort_values(by = 'date_created', inplace = True, ignore_index = True)
bert_data[bert_data['long_text'].isna()]

,text_type,ID,date_created,year,long_text


In [22]:
#remove custom stopwords
with open("../Data/custom_stop_words.txt", 'r') as file:
    stopwords = [line.strip() for line in file]
    
print (stopwords[:5])

['physic', 'denizen', 'grako', 'redfield', 'womp']


In [27]:
if 'dubai' in updated_stopwords:
    print('yes')

yes


In [23]:
#since text is unprocessed, include uppercase and all-caps version of each word
updated_stopwords = []

for word in stopwords:
    updated_stopwords.extend([word, word.upper(), word.capitalize()])

In [24]:
#remove custom stopwords from long_text

bert_data['long_text'] = bert_data['long_text'].apply(lambda text: 
                                                      ' '.join([word for word in text.split() if word not in updated_stopwords]))

In [25]:
bert_data

,text_type,ID,date_created,year,long_text
0,comment,c5c54q4,2012-07-11 00:50:58,2012,That dessert's a rich for me.
1,comment,c5edn0u,2012-07-15 21:59:34,2012,"""A SILVER one?! I YOU DAD!"""
2,comment,c6d18gk,2012-09-25 07:57:13,2012,Yet i stared at the for a 45 seconds to see if...
3,comment,c6d1cs1,2012-09-25 08:04:04,2012,seriously?
4,comment,c6d2fss,2012-09-25 09:13:23,2012,"[FYSR] = from your sister subreddit. IMO, its ..."
...,...,...,...,...,...
127681,comment,joztxc3,2023-06-21 22:45:47,2023,that’s disappointing
127682,comment,jozujeb,2023-06-21 22:49:54,2023,India also has fees and some other benefits fo...
127683,comment,jozw2h1,2023-06-21 23:00:05,2023,> struggles that during you're not special lol...
127684,comment,jozwyd6,2023-06-21 23:06:11,2023,🤣🤣


In [26]:
bert_data.to_csv('../bert_model_data/bert-data-sw.csv')

In [29]:
for row in bert_data['long_text']:
    if 'dubai' in row.split():
        print ('yes')